In [67]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from dotenv import load_dotenv
from pdf2image import convert_from_path, convert_from_bytes
import time
import os


In [68]:
download_directory = '/home/mayk/projetos/Sei'

# Configurar as opções do Chrome
options = webdriver.ChromeOptions()

# Configurar o local de download
prefs = {
    'download.default_directory': download_directory,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': False
}
options.add_experimental_option('prefs', prefs)


In [69]:
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)
load_dotenv()
browser.maximize_window()
senha = os.getenv("SENHA")
senha1 = os.getenv("SENHA1")
wait = WebDriverWait(browser, 3)

browser.get(
    "https://sipe.dpu.def.br/sip/login.php?sigla_orgao_sistema=DPU&sigla_sistema=SEI")


def format_string(text):
    if (len(text) > 1):
        formatted_text = text.title()
        last_char = formatted_text[-1]
        if last_char == "," or last_char == ".":
            formatted_text = formatted_text[:-1]
        return formatted_text
    return ""

def remove_before_space(string):
    index = string.find(" ")
    if index == -1:
        return string
    else:
        return string[index+1:]

In [70]:
browser.find_element(By.ID, "txtUsuario").send_keys("maykon.soares")
browser.find_element(By.ID, "pwdSenha").send_keys(senha)
Select(browser.find_element(By.ID, "selOrgao")).select_by_value("8038")
browser.find_element(By.ID, "Acessar").click()
element = wait.until(EC.element_to_be_clickable(
    (By.CLASS_NAME, "sparkling-modal-close")))

element.click()
browser.find_element(By.CLASS_NAME, "ancoraPadraoPreta").click()
elements = browser.find_elements(By.CLASS_NAME, "processoVisualizado")
n = 0


In [71]:
for element in elements:
    element.send_keys(Keys.CONTROL + Keys.RETURN)
    janelas = browser.window_handles
    browser.switch_to.window(janelas[1]) 

    ActionChains(browser)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .perform()

    iframe = wait.until(EC.presence_of_element_located((By.ID, "ifrVisualizacao")))
    browser.switch_to.frame(iframe)

    iframe = wait.until(EC.presence_of_element_located((By.ID, "ifrArvoreHtml")))
    browser.switch_to.frame(iframe)


    value = browser.find_element(By.CLASS_NAME, "Texto_Justificado_Recuo_Primeira_Linha")
    text_content = value.text

    parts = text_content.split(", ", 1)

    if len(parts) > 1:
        result = parts[1]
    else:
        result = ""
        
    result = format_string(result)
    print(result)
    
    browser.switch_to.default_content()
    browser.switch_to.default_content()
    
    ActionChains(browser)\
        .send_keys(Keys.ARROW_UP)\
        .send_keys(Keys.ARROW_UP)\
        .send_keys(Keys.ARROW_UP)\
        .send_keys(Keys.ARROW_UP)\
        .perform()

    iframe = wait.until(EC.presence_of_element_located((By.ID, "ifrVisualizacao")))
    browser.switch_to.frame(iframe)
    
    Element = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//*[@id='divArvoreAcoes']/a[16]")))
    Element.click()
    
    # ActionChains(browser)\
    #     .key_down(Keys.CONTROL)\
    #     .send_keys("a")\
    #     .key_up(Keys.CONTROL)\
    #     .send_keys(Keys.BACKSPACE)\
    #     .perform()
    
    # ActionChains(browser)\
    #     .send_keys(result)\
    #     .perform()
        
    browser.find_element(By.NAME, "sbmRegistrarAnotacao").click()
    
    browser.execute_script("window.open('https://www3.riopro.com.br/dpu.patrirp/principal.rpro?opcao_menu=128&visao=BENS&ordem=bn_dtope%20desc&hab=5474', '_blank')")
    janelas = browser.window_handles
    browser.switch_to.window(janelas[2]) 
    browser.implicitly_wait(8)
    
    
    browser.find_element(By.CLASS_NAME, "with_background")
    
    ActionChains(browser)\
        .send_keys(Keys.TAB)\
        .send_keys("MAYKON.SOARES")\
        .send_keys(Keys.TAB)\
        .send_keys(senha1)\
        .send_keys(Keys.TAB)\
        .send_keys(Keys.TAB)\
        .send_keys(Keys.ENTER)\
        .perform()
        
    rel = browser.find_elements(By.CLASS_NAME, "menuButton")
    rel[6].click()
    
    radio_button = browser.find_element(By.XPATH, "//input[@type='radio' and @value='249']")
    
    browser.execute_script("arguments[0].checked = true;", radio_button)
    
    browser.find_element(By.ID, "botao_filtros").click()
    
    time.sleep(1) 
    
    janelas = browser.window_handles
    browser.switch_to.window(janelas[3]) 
    browser.implicitly_wait(3)
    
    check_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and @value='bn_local']")))
    check_button.click()

    text = remove_before_space(result)
    print(text)
    
    time.sleep(1) 
    
    actions = ActionChains(browser)
    actions.send_keys(Keys.ARROW_DOWN)
    actions.send_keys(Keys.TAB)
    actions.perform()
    
    time.sleep(1) 
    
    actions = ActionChains(browser)
    actions.send_keys(text)
    actions.perform()
    
    time.sleep(2)
    
    click = browser.find_elements(By.CLASS_NAME, "select2-results__option")
    click[1].click()
    
    browser.find_element(By.ID, "botao_   OK   ").click()
    janelas = browser.window_handles
    browser.switch_to.window(janelas[2]) 
    browser.implicitly_wait(8)

    browser.find_element(By.ID, "botao_visualizar pdf").click()

    pages = convert_from_path('relatorioriopro.pdf')

    for i in range(len(pages)):
        pages[i].save(text.replace(" ", "-") + "-" + str(i) + '.jpg', 'JPEG')
    
    break


Dra. Vanessa Bongiolo Brogni
Vanessa Bongiolo Brogni
